<a href="https://colab.research.google.com/github/aidanODnnll/Isofragmented_Reactions/blob/main/IsofragmentedReactions_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimation of Enthalpy of Formation
 
This jupyter notebook implements capabilities to estimate enthalpy of formation of a target molecule using isofragmented reactions. If you are using this for educational or research purposes, please cite the following publication:
 

 
## Needed Libraries
 
Numpy and Scipy will be needed for data analysis. Pyomo will be needed in order to create the MILP model. RDkit is required in order to count the features of an molecule. Additionally a solver will be required for the MILP model, this notebook allows for the user to access GLPK, or CPLEX through the use of NEOS. If the user has thier own solver with license they would like to use such as Gurobi or CPLEX they can run this file localy on their own computer. 



In [ ]:
import pandas as pd
import scipy.stats
import numpy as np
import os
!pip install pyomo
from pyomo.environ import *

## Import rdkit
!curl -L bit.ly/rdkit-colab | tar xz -C /
from rdkit import Chem

from urllib.request import urlopen
url = 'https://drive.google.com/uc?id=1zAKMev2bo6STdgAkLzAiwGcFfKgUBZ0k'
py = urlopen(url).read().decode('utf-8')
exec(py)

     |████████████████████████████████| 8.9MB 35.5MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   3622      0 --:--:-- --:--:-- --:--:--  3622
100   133  100   133    0     0   1385      0 --:--:-- --:--:-- --:--:--  1385
100   620  100   620    0     0   3757      0 --:--:-- --:--:-- --:--:--  3757
100 29.6M  100 29.6M    0     0  22.2M      0  0:00:01  0:00:01 --:--:-- 27.5M


## Starting the Code 
 
The function that the user will call in order to create the estimates. 2 text files must be passed when the function is called. The first is a file of known molecules and their known enthalpy of formation, the second is a list of unknown molecules. If the user does not wish to use the provided NIST database they can chose to input `UseNIST = False`. The NIST database provided has molecules made up of C, O, and H. If the user provides known molecules this model can handle molecules made up of C, O, H, N, and F atoms. 
 

In [ ]:

def run(UnknownMoleculesTxt, KnownMoleculesTxt = [], UseNIST = True):
    NumberOfRxns = 40
    StoichLimit = 5
    f = open(KnownMoleculesTxt)
    if len(f.readlines()) >= 1:
        EnthalpyOfFormation = keyMaker(KnownMoleculesTxt)
        KnownSMILES = justSMILES(KnownMoleculesTxt)
        KnownMolecules = KnownSMILES.splitlines()
        String_KMtF, values = file_transform(KnownSMILES,atoms, mode = 'pathway fps')
        KnownMoleculesToFeatures = CreateData(String_KMtF)
    else: 
        EnthalpyOfFormation = {}
        KnownSMILES = []
        KnownMolecules = []
        KnownMoleculesToFeatures = {}
    f.close()
    f = open(UnknownMoleculesTxt)
    if len(f.readlines()) < 1:
        return 'Error: No Molecules with Unknown Enthalpy Provided'
    UnknownMolecules = open(UnknownMoleculesTxt, 'r').readlines()
    String_UMtF, values = file_transform(open(UnknownMoleculesTxt).read(),atoms, mode = 'pathway fps') 
    UnknownMoleculesToFeatures = CreateData(String_UMtF)
    features = values[2:]
    GATestimates = values[:2]
    if UseNIST: 
        NISTKnownFile = 'https://drive.google.com/uc?id=1KSSbE9eX4__faxWEG9PA1z6vgH3r-7Wf'
        NISTEnthalpy = keyMaker(NISTKnownFile)
        KnownMolecules += justSMILES(NISTKnownFile).splitlines()
        KnownMolecules = list(set(KnownMolecules))
        f = open(NISTKnownFile)
        NISTString, values = file_transform(f.read(),atoms, mode = 'regular')
        f.close()
        print(CreateData(NISTString))
        KnownMoleculesToFeatures.update(CreateData(NISTString))
        EnthalpyOfFormation.update(NISTEnthalpy)
    Weights = {}
    for f in features: 
        if len(f) == 1: 
            Weights.update({f:0})
        else:
            Weights.update({f:1})
    for e in GATEstimates:
        Weights.update({e:1000})
    solutions = 'Molecule \t Estimated Enthalpy of Formation (kcal/mol) \t Uncertainty \n'
    for mol in UnknownMolecules:
        if mol in KnownMolecules:
            solutions += str(mol) +'\t' + str(EnthalpyOfFormation[mol]) + '\t Enthalpy already known \n'
        else:
            results = 'chosen = '+ str(mol) + '\n'
            rxns,n = findRxns(UnknownMoleculesToFeatures, Weights, NumberOfRxns, StoichLimit, KnownMolecules, KnownMoleculesToFeatures, values, mol)
            results += rxns
            Estimate, dHList = determineEnthalpy(results, n, EnthalpyOfFormation, KnownMoleculesToFeatures, UnknownMoleculesToFeatures)
            Mean, Error = findAverages(Estimate, dHList)
            solutions += str(mol) +'\t' + str(Mean) + '\t' + str(Error) + '\n'
    return solutions



For this example `run()` will be broken up and its functions will be called individually in order to demonstrate how they work.
 
To simplify the demonstration of the code the NIST database will not be used. Instead a selecion of 20 molecules will be treated as the known and there will only be one unknown molecule. The first step is running `file_transform()` which counts the features for each molecule along with providing some estimates.


In [ ]:
def justSMILES(filename):
    lines = urlopen(filename).read().decode('utf-8').splitlines()
    SMILES = ''
    for l in lines:
        l = l.split()
        SMILES += l[0] + '\n'
    return SMILES

UnknownMoleculesTxt = 'https://drive.google.com/uc?id=1_UCgLfYOzOlJawFurxgRZpCGjvMR-ne_'
KnownMoleculesTxt = 'https://drive.google.com/uc?id=1ejo5R3COY2Yr10zyMwz9-iuxHcidaEy4'

atoms = ['C','O','H'] 
KnownSMILES = justSMILES(KnownMoleculesTxt)
String_KMtF, values = file_transform(KnownSMILES,atoms, mode = 'regular') #'pathway fps' is the mode used in the paper  
String_UMtF,values = file_transform(urlopen(UnknownMoleculesTxt).read().decode('utf-8'),atoms, mode = 'regular') #'pathway fps' is the mode used in the paper 
features = values[2:]
GATEstimates = values[:2]

print('Example of what file_transform() returns: \n' + String_UMtF)

Example of what file_transform() returns: 
SMILES                                                                          atomization_energy  uncertainty                       C              O              H             CO            C=O             CC            C=C            C#C           C[H]           c[H]             cC           O[H]             cc             cO             co
CCCCOC                                                                                      -1597.78                0.22              5              1             12              2              0              3              0              0              0              0              0              0              0              0              0



Note that in this example, the molecule `CCCCOC` is being treated as if it has an unknown enthalpy and will be the target molecule. Multiple molecules can have their enthalpy marked as unknown.

The information provided by the user along with what `file_transform()` obtained must be converted from strings to data types useable by the MILP model. This is done by a couple functions in the next cell.

In [ ]:
def keyMaker(filename):
    f = np.loadtxt(filename, dtype=object, comments = None)
    key = {}
    for l in f:
        key.update({l[0]:float(l[1])})
    return key

def CreateData(string):
    l = string.splitlines()
    z = 0 
    res = []
    #This for loop puts the data into a list to be used later 
    for x in l:
        z += 1
        # res += [delspace(x)]
        res += [x.split()]
    MolToFactors = MakeDataDict(res[0],res[1:])
    return MolToFactors

def MakeDataDict(features,data):
    res = {}
    newf = features[1:]
    mols = []
    z = 0
    #Goes through each index of data
    for l in data:
        mol = l[0] 
        mols += [mol]
        point = 1 
        # assigns the value to its corresponding feature for each molecule      
        for i in newf:
            res.update({(mol,i):float(l[point])})
            point += 1
    return res


KnownMolecules = KnownSMILES.splitlines()
UnknownMolecules = urlopen(UnknownMoleculesTxt).read().decode('utf-8').splitlines()
KnownMoleculesToFeatures = CreateData(String_KMtF)
UnknownMoleculesToFeatures = CreateData(String_UMtF)
EnthalpyOfFormation = keyMaker(KnownMoleculesTxt)


# print('List of Molecules with Known Enthalpy:', KnownMolecules) Dont need bc given
print('Molecules with Unknown Enthalpy:', UnknownMolecules)
print('List of Features:', features)
# print('Dictionaries connecting molecules to features', KnownMoleculesToFeatures, UnknownMoleculesToFeatures)
print('Known Enthalpy of Formations:', EnthalpyOfFormation)

Molecules with Unknown Enthalpy: ['CCCCOC']
List of Features: ['C', 'O', 'H', 'CO', 'C=O', 'CC', 'C=C', 'C#C', 'C[H]', 'c[H]', 'cC', 'O[H]', 'cc', 'cO', 'co']
Known Enthalpy of Formations: {'C[CH]C': 21.51, 'CC(=O)O': -103.49, 'CCCCCC': -39.96, 'CCOC=O': -95.12, 'C=[CH]': 71.46, 'CCC(=O)CC(=O)C': -105.09, 'O=C1CC2C(C1)(C)CCCC2': -65.77, 'CCCCCCCCCC1CCCCC1': -75.57, 'CCCC=CC': -11.23, 'COCCOC': -81.93, 'CCCCC': -35.16, 'C1CCCO1': -44.02, 'C1CCO1': -19.25, 'COC=O': -86.52, 'OCC(C)C': -67.61, 'CC(C)(C)C': -39.67, 'C=O': -25.96, 'CC=CC=O': -25.55, 'C1CC2C1C3C2C3': 56.17}


Next the features must be assigned a weight based on how important they are to conserve in an isofragmented reaction. The estimate of atomization energy and uncertainty are not important to the reaction so they are assigned a very high value. If the length of a feature is 1, it is assumed to be an element and given a value of 0 since it must be conserved. Everything else is assigned a value of 1.

In [ ]:
Weights = {}
for f in features: 
    if len(f) == 1: 
        Weights.update({f:0})
    else:
        Weights.update({f:1})
for e in GATEstimates:
    Weights.update({e:1000})
print(Weights)



{'C': 0, 'O': 0, 'H': 0, 'CO': 1, 'C=O': 1, 'CC': 1, 'C=C': 1, 'C#C': 1, 'C[H]': 1, 'c[H]': 1, 'cC': 1, 'O[H]': 1, 'cc': 1, 'cO': 1, 'co': 1, 'atomization_energy': 1000, 'uncertainty': 1000}


## Creating the Isofragmented Reactions

The next step is to loop through the molecules with unknown enthalpy in order to create reactions and then estimates for each molecule, in this example there is only 1 molecule so the loop will not be particularly necessary. 

First the molecule along with all the data will be passed into the function `findRxns()` which will create the MILP model and solve to find a number of isodesmic reactions. The MILP model also makes use of several other functions such as `StoichRule1()`, `StoichRule2()`, and `MakeIso()` to create constraints. In order to solve the MILP model the program must make use of a solver such as CPLEX, Gurobi and ... The user must specify the solver using `solver = SolverFactory('Specified_solver)` and then add tolerances and time limits using `solver.options('tolerance/time')` where the user must look at solver documentation to determine the name of the option. For this example GLPK is used because the model is not very large, if the user wants to run this with more molecules they should use CPLEX provided by NEOS or their own solver. The solver settings for NEOS, GLPK, Gurobi, and CPLEX are provided all provided. 
This particular example uses GLPK because it is very simple, NEOS is the recomened solver to use.



In [ ]:
# Uncomment if you want to use GLPK
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [ ]:

# os.environ['NEOS_EMAIL'] = 'user@email' # Replace with your own email, if you want to run CPLEX through NEOS 

def StoichRule1(m, i):
    return -1*m.n*m.y[i] + m.z[i] <= m.s[i]

def StoichRule2(m, i):
    return   -1*m.y[i] + 1*m.n*m.z[i] >= m.s[i]

def OneSRule(m, i):
    return m.y[i] + m.z[i] <= 1

def slack(m,k):
    sums = sum(m.a[i,k]*m.s[i] for i in m.mol)
    return sums <= m.c[k]

def slack2(m,k):
    sums = sum(m.a[i,k]*m.s[i] for i in m.mol)
    return sums >= -1*m.c[k]

def makeEta(m,i):
    return -1*m.eta[i] <= m.s[i]

def makeEta2(m,i):
    return m.eta[i] >= m.s[i]

def MakeIso(m):
    sums = sum(m.c[i]/m.w[i] for i in m.factors if  m.w[i] != 0 and m.w[i] <= 100  )
    return sums <= m.Lambda

def findRxns(UnknownMoleculesToFeatures, Weights, NumberOfRxns, maxStoich, KnownMolecules, KnownMoleculesToFeatures, features, mol):   
    d = KnownMoleculesToFeatures
    newfactors = {}
    #Adds Target molecule to the dictionary of features and list of molecules
    for x in features:
        z = UnknownMoleculesToFeatures[(mol,x)]
        d.update({('target', x):z})
    KnownMolecules += ['target']
    targetMol = 'target'
    m = ConcreteModel()
    # sets
    m.mol = Set(initialize = KnownMolecules) # the molecules
    m.factors = Set(initialize = features)# factors
    #parameters
    m.a = Param(m.mol, m.factors, initialize = d)# gives values of factors for molecules
    m.w = Param(m.factors, initialize = Weights) # gives weights of factors
    m.n = Param(initialize = maxStoich, mutable = True) # Max stoich coeff
    m.Lambda = Param(initialize = 0, mutable = True)

    #variables 
    m.y = Var(m.mol, initialize = 0, within = Binary)# Whether it is a reactant
    m.z = Var(m.mol, initialize = 0, within = Binary)# whether it is a product
    m.s = Var(m.mol, initialize = 0, within = Reals)# stoich coeff
    m.c = Var(m.factors, initialize = 0, within =NonNegativeReals) # This is the amount of extra bonds needed
    m.eta = Var(m.mol, initialize = 0, within =NonNegativeReals) # Positive value of S
    # con
    m.chosen = Constraint( expr = m.z[targetMol] >= 1) # Ensures target molecule is in all reactions 
    m.StoichLim1 = Constraint(m.mol, rule = StoichRule1) # Makes sure we don't go over the stoich limit
    m.StoichLim2 = Constraint(m.mol, rule = StoichRule2)
    m.OneSide = Constraint(m.mol, rule = OneSRule) # Make sure the molecule only apears on one side
    m.slack = Constraint(m.factors, rule = slack)# Determines m.c
    m.slack2 = Constraint(m.factors, rule = slack2) 
    m.isoD = Constraint(rule = MakeIso)#Makes the reaction isofragmented
    m.posStoich =  Constraint(m.mol, rule = makeEta) # Determines m.eta
    m.posStoich2 =  Constraint(m.mol, rule = makeEta2)
    m.constL = ConstraintList()
    # Ensuring that elements must be conserved 
    expr = 0
    for j in m.w:
        if value(m.w[j]) == 0:
            expr += m.c[j]
    m.constL.add( expr == 0 )
    
    #objective
    m.Objective = Objective(expr=sum(m.eta[i]*m.a[i,'uncertainty'] for i in m.mol), sense = minimize)
    
    #Solver options 
    # Uncomment to use GLPK
    solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
    
    # Uncomment to use NEOS
    # solver = SolverManagerFactory('neos')

    # Uncomment to use Gurobi
    # solver = SolverFactory('gurobi')
    # solver.options['MIPGapAbs'] = (.1)
    # solver.options['TimeLimit'] = 250
    
    # Uncomment to use CPLEX
    # solver = SolverFactory("cplex")
    # solver.options['mip_tolerances_absmipgap'] = .1
    # solver.options['timelimit'] = 250
    
    sol = ''
    t = 1
    # iterate through reactions 
    for i in range(NumberOfRxns):
        # Uncomment if using any solver but NEOS
        results = solver.solve(m,load_solutions=False)

        # Uncomment if using NEOS
        # results = solver.solve(m, opt = 'cplex', load_solutions=False)

        # Check solver status 
        if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
            m.solutions.load_from(results)

            ## Options to see the results as displayed by the solver
            # print(results) 
            # print(m.display())
            
            # Turn the solution into a more understandable format
            res = ''
            resC = 0
            res2C = 0
            res2 = ''
            for v in m.component_objects(Var, active=True):
                if str(v) == 's':
                    for i in v:
                        if abs(value(v[i])) >= .01:
                            if value(v[i]) <= 0:
                                if resC >= 1:
                                    res += ' + '
                                res += str(round(abs(value(v[i])),3)) + '*' + str(i) 
                                resC += 1
                            if value(v[i]) >= 0:
                                if res2C >= 1:
                                    res2 += ' + '
                                res2 += str(round(abs(value(v[i])),3)) + '*' + str(i) 
                                res2C += 1
            sol += 'Reaction ' + str(t) + ':' + '      ' +  res + ' <--> ' + res2 + ' \n'
       
        # Possible failure cases 
        elif (results.solver.termination_condition == TerminationCondition.infeasible):
            break
        elif results.solver.status == SolverStatus.aborted:
            break
        # print(t) # Allows user to keep track of what reaction number the model is on (not necessary for use)
        t += 1
        expr = 0
        ### Adjusted Integer cuts, uncomment else statements for traditional integer cuts(not recommended)
        for j in m.y:
            if value(m.y[j]) == 1:
                expr += (1-m.y[j])
            #else:
                #expr += m.y[j]
        for r in m.z:
            if value(m.z[r]) == 1:
                expr += (1-m.z[r])
            #else:
                #expr += m.z[r]
        m.constL.add( expr >= 1 )
    return(sol,t-1)

# This will clean the results into a easily readable format
def MakeResClean(results):
    CleanRes = ''
    target = results.splitlines()[0].split()[2]
    for l in results.splitlines()[1:]:
        l = l.split()
        CleanRes += l[0]+ ' ' + l[1] + ' '
        for i in l[2:]:
            if i == '+' or i == '<-->':
                CleanRes += ' ' + i + ' '
            else:
                GettingStoich = True
                stoich = ''
                mol = ''
                Searching = True
                x = 0
                while Searching:
                    if GettingStoich:
                        if i[x] == '*':
                            if abs(float(stoich) - int(float(stoich))) < .01:
                                  stoich = int(float(stoich))
                            else:
                                  stoich = float(stoich) 
                            GettingStoich = False
                
                        else:
                            stoich += i[x]
                    else:
                        mol = i[x:]
                        if mol == 'target':
                            mol = target
                        if stoich == 1:
                            CleanRes += mol + ' '
                        else:
                            CleanRes += str(stoich) + ' ' + mol + ' '
                        Searching = False
                    x += 1
        CleanRes += '\n'
    return CleanRes

# Example
NumberOfRxns = 10
StoichLimit = 5
mol = UnknownMolecules[0]
results = 'target = '+ str(mol) + '\n'
rxns,n = findRxns(UnknownMoleculesToFeatures, Weights, NumberOfRxns, StoichLimit, KnownMolecules, KnownMoleculesToFeatures, values, mol)
results += rxns
print(MakeResClean(results))


Reaction 1: CCCCCC  + COCCOC  <--> 2 CCCCOC 
Reaction 2: 2 CCCCC  + COC=O  <--> CCCCCC  + C=O  + CCCCOC 
Reaction 3: CCCCC  + 2 COC=O  <--> CCOC=O  + C=O  + CCCCOC 
Reaction 4: CCCCC  + C1CCO1  + COC=O  <--> C1CCCO1  + C=O  + CCCCOC 
Reaction 5: COC=O  + 2 CC(C)(C)C  <--> CCCCCC  + C=O  + CCCCOC 
Reaction 6: CCOC=O  + OCC(C)C  <--> CC(=O)O  + CCCCOC 
Reaction 7: CCOC=O  + 3 CCCCC  <--> 2 CCCCCC  + C=O  + CCCCOC 
Reaction 8: CCOC=O  + COCCOC  + CCCCC  <--> COC=O  + 2 CCCCOC 
Reaction 9: CCOC=O  + CCCC=CC  <--> CC=CC=O  + CCCCOC 
Reaction 10: COCCOC  + CCCCC  + C1CCCO1  <--> C1CCO1  + 2 CCCCOC 



## Estimating Enthalpy of Formation 

After the reactions have been created the next step is to calculate the enthalpy of formation for the target molecule along with the uncertainty for each reaction. This is done using a number of functions; `determineEnthalpy()` breaks the results down into their individual reactions, `delspace()` turns a string into a list of its components, `calcHrxn()` determines the enthalpy of reaction using the estimated values, and finally `Calculator()` will determine the values. 

In [ ]:

def determineEnthalpy(rxns, n, KnownEnthalpy, KnownMoleculesToFeatures, UnknownMoleculesToFeatures):
    lines = rxns.splitlines()
    rxnCount = 0
    listR = []
    res = [] 
    dHList = []
    foundRxn = False
    for l in lines: 
        if l[:6] == 'target':
            foundRxn = True
            # target = delspace(l[8:])[0]
            # listR += delspace(l[8:])
            target = l[8:].split()[0]
            listR += l[8:].split()
            pass
        elif foundRxn == True:
            # listR += [delspace(l[13:])]
            listR += [l[13:].split()]
            rxnCount += 1
            if rxnCount == n:
                Estimate, dHList = Calculator(KnownEnthalpy,listR, n, \
                                              KnownMoleculesToFeatures, UnknownMoleculesToFeatures)
                foundRxn = False 
    return Estimate, dHList

def calcHrxn(listMol, stoich,chosenStoich, target, d, UnknownMoleculesToFeatures):
    Hrxn = sum(stoich[m]*d[(m,'atomization_energy')]  for m in listMol)\
     + chosenStoich*UnknownMoleculesToFeatures[(target,'atomization_energy')]
    return Hrxn  

def Calculator(KnownEnthalpy, listR, numRxn, KnownMoleculesToFeatures, UnknownMoleculesToFeatures):
    d = KnownMoleculesToFeatures
    target = listR[0]
    Estimate = []
    HrxnSum = 0
    dHrxnList = []
    for l in range(numRxn+1):
        #First make list of all molecules involved and a dictionary linking them to their coeff
        listMol = []
        stoich = {}
        reactant = True
        targetStoich = 0
        if l == 0:
            pass
        else:
            stillLooking = True
            for i in listR[l]:
                coeff = ''
                coeffStatus = True
                currentMol = ''  
                if i == '<-->':
                    reactant = False
                else: 
                    for k in range(len(i)):
                        if stillLooking == False:
                            break
                        if (i[k].isnumeric() or i[k] == '.') and coeffStatus:
                            coeff += i[k] 
                        elif i[k] == '*':
                            coeffStatus = False
                        elif i[k] == '+':
                            break
                        elif i[k:] == 'target':
                            if reactant:
                                targetStoich = float(coeff)*-1
                            else:
                                targetStoich = float(coeff)
                            stillLooking = False
                            break
                        else:
                            currentMol += i[k]    
                    if currentMol != '':
                        listMol += [currentMol]
                        if reactant:
                            stoich.update({currentMol:float(coeff)*-1})
                        else:
                            stoich.update({currentMol:float(coeff)})
            if  targetStoich != 0:
                Hrxn = calcHrxn(listMol, stoich,targetStoich, target, d, UnknownMoleculesToFeatures)
                Estimate += [(Hrxn - sum(KnownEnthalpy[m]*stoich[m] for m in listMol)/targetStoich)]
                dHrxnsqr = sum(abs(stoich[m])*d[(m,'uncertainty')]**2 for m in listMol) \
                + abs(targetStoich)*UnknownMoleculesToFeatures[(target,'uncertainty')]**2
                dHrxn = dHrxnsqr**.5/abs(targetStoich)
                dHrxnList += [dHrxn]
    return Estimate, dHrxnList
n = 10
Estimate, dHList = determineEnthalpy(results, n, EnthalpyOfFormation,\
                                     KnownMoleculesToFeatures, UnknownMoleculesToFeatures)
print('Estimates from each Reactions:', Estimate)
print('Uncertinaty from each Reaction:', dHList)
 



Estimates from each Reactions: [-63.244999999999735, -67.17999999999972, -64.95999999999967, -58.650000000000034, -72.07999999999987, -61.49000000000002, -69.39999999999938, -63.56499999999935, -69.97000000000008, -61.78999999999968]
Uncertinaty from each Reaction: [0.3183158808479401, 0.6073713855624086, 0.7114773362518303, 0.7109149034870489, 0.7211795892841116, 1.09754270987511, 0.6509992319503918, 0.3945250308915773, 1.0092571525632108, 0.39427148007432644]


The final step is to remove outliers and obtain an average value over all 
the reactions before returning the results. 


In [ ]:
def findAverages(Estimate, dHlist):
    MultH = 0
    n = len(Estimate)
    for l in range(n):
        MultH += np.log10(float(Estimate[l]) + 10000)
    logM = MultH/n
    standardDev = np.std(Estimate)
    mean = 10**logM - 10000
    correctedMult = 0
    cN = 0 
    if len(Estimate) == 1:
        cMean = listH[0]
    else:
        for x in Estimate:
            if mean - 2*standardDev <= x <= mean + 2*standardDev:
                correctedMult += np.log10(x + 10000) 
                cN += 1
            elif standardDev == 0:
                correctedMult += np.log10(x + 10000) 
                cN += 1
        clogM = correctedMult/cN
        cMean = 10**clogM - 10000
        meanE = np.mean(dHlist)
        stdevE = np.std(dHlist)
        df = n-1
        lower, upper = scipy.stats.t.interval(.90, df, meanE, stdevE)
    return(cMean, upper)

Mean, Error = findAverages(Estimate, dHList)
header = ['Molecule', 'Estimated Enthalpy of Formation (kcal/mol)', 'Uncertainty (kcal/mol)',  'Known Value (kcal/mol)']
solution = [str(mol), str(Mean),str(Error),'-61.69' ]
print(pd.DataFrame(solution,header))

                                                             0
Molecule                                                CCCCOC
Estimated Enthalpy of Formation (kcal/mol)  -65.23383792684035
Uncertainty (kcal/mol)                      1.1042249910046134
Known Value (kcal/mol)                                  -61.69
